In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re

Fil = pd.read_pickle('../data/raw/Fil.pkl')
Dokument = pd.read_pickle('../data/raw/Dokument.pkl')
Sag = pd.read_pickle('../data/raw/Sag.pkl')
SagDokument = pd.read_pickle('../data/raw/SagDokument.pkl')

This notebook identifies the Files that are not relevant for the model
These are files that are duplicates in some way. Either by :
1. Being the question for which the answer file contains the question
2. Being an intermediate file, where a final answer exists
3. Being an EU file, where a subsequent national file exists

The notebook then extracts a random subset in order to fine tuen a BERT emebedding model

In [2]:
Dokument['kategoriid'].value_counts()[0:20]
#man kan nok godt fjerne udvalgsspørsmål (36), samrådsspørgsmål (37),  foreløbigt svar (30) og  §20 spørgsmål (14)
#det sparer 245123 +23212+ 53136 + 15763 = 335834 = ca. 40% af dokumenterne
#man kan også fjerne alle dokumenter relateret til sager med typeid 7, da disse er EU sager

C:\Users\rune7\AppData\Local\Temp\ipykernel_20508\3150753189.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Dokument['kategoriid'].value_counts()[0:20]


22    280349
36    245123
15     81565
14     53137
53     43353
30     23212
56     21811
55     20479
37     15763
51     11413
31      8886
23      4311
46      4306
58      4011
35      3572
24      3451
18      3245
25      3190
65      2787
11      2750
Name: kategoriid, dtype: int64

In [3]:
len(Dokument)

871023

In [3]:
len(Fil)

491215

In [4]:
unwanted_categories = [36,37,30,14]

In [5]:
Not_EU_Sag = Sag[Sag['typeid'] != 7]
Dokument_ids = SagDokument[SagDokument['sagid'].isin(Not_EU_Sag['id'])]['dokumentid'].unique()
len(Dokument_ids)

541969

In [6]:
Filtered_dok = Dokument[~Dokument['kategoriid'].isin(unwanted_categories)]
len(Filtered_dok)

533788

In [7]:
Filtered_dok = Filtered_dok[Filtered_dok['id'].isin(Dokument_ids)]
len(Filtered_dok)

359596

In [8]:
Filtered_fil = Fil[Fil['dokumentid'].isin(Filtered_dok['id'])]
len(Filtered_fil)

312751

In [9]:
#pickle the filtered dataframes
Filtered_dok.to_pickle('../data/raw/Filtered_dok.pkl')
Filtered_fil.to_pickle('../data/raw/Filtered_fil.pkl')

In [10]:
#taking dokument have a spørgsmålsid that is not none and a kategoriid == 22
question_doks = Filtered_dok[~Filtered_dok['spørgsmålsid'].isna() & (Filtered_dok['kategoriid'] == 22)]['titel'].tolist()
question_ids = Filtered_dok[~Filtered_dok['spørgsmålsid'].isna() & (Filtered_dok['kategoriid'] == 22)]['id'].tolist()

In [11]:
filtered_questions = []
import re 
pattern = r'^((Svar på spm\. nr\. S \d+:)|(\w+ - svar på spm\. \d+)|Svar på KOM \(\d+\) \d+ spm\. \d+,?\s+om\s+|(Aktstk\.? \d+ - svar på spm\. \d+)|([A-Z]{3} \(\d+\) \d+ - svar på spm\. \d+)|(svar på spm\. nr\. S \d+:)|(L \d+ - svar på spm\. \d+)|(Svar på spm\.nr\. S \d+:)|([A-Z]{3} alm\. del - svar på spm\. \d+)|(UUI alm\. del - svar på spm\. \d+)|(Svar på spm\. nr\. \d+:)|([A-Z]{3} alm\. del - svar på spm\. \d+)|(URU alm\. del - svar på spm\. \d+)|(B \d+ - svar på spm\. \d+)|(KEB alm\. del - svar på MFU spm\. \d+)|(TRU alm\. del - svar på spm\. \d+))'

for question in question_doks:
    filtered_question = re.sub(pattern, '', question)
    filtered_questions.append(filtered_question)



In [12]:
zip_iterator = zip(question_ids, filtered_questions)
question_dict = dict(zip_iterator)

In [25]:
import random
random_subset = pd.read_pickle('../data/raw/random_subset.pkl')
random_subset2 = pd.read_pickle('../data/raw/random_subset2.pkl')



In [37]:
#generating a new random subset, which is different from the one here
random_subset3 = random.sample(question_dict.keys(), 40000)


C:\Users\rune7\AppData\Local\Temp\ipykernel_20508\4071157150.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_subset3 = random.sample(question_dict.keys(), 40000)


In [38]:
#we tak the keys from random subset 1 and 2 and make it a list
random_subset1_keys = [i[0] for i in random_subset]
random_subset2_keys = [i[0] for i in random_subset2]


In [39]:
#removing all the questions that are already in the random subset or random subset 2
random_subset3 = [x for x in random_subset3 if x not in random_subset1_keys]
random_subset3 = [x for x in random_subset3 if x not in random_subset2_keys]
len(random_subset3)

31415

In [40]:
#get the ids and  question_doks belonging to the random_subset3
random_subset3_questions = []
for id in random_subset3:
    random_subset3_questions.append((id,question_dict[id]))


In [42]:

#save as pickle
import pickle
with open('../data/raw/random_subset3.pkl', 'wb') as f:
    pickle.dump(random_subset3_questions, f)


In [44]:
#doublecheck that random_subset2_questions is not the same as random_subset
count = 0
for id in random_subset:
    for id2 in random_subset3_questions:
        if id[0] == id2[0]:
            count+=1
            print(count)